In [2]:
## for spatial analysis
import geopandas as gpd

## for data analysis
import pandas as pd

## for pretty charts
import plotly.express as px

# for plotly themes
import plotly.io as pio

## for URL requests
import urllib.request
import requests

## for maps
import folium
from folium import plugins

In [5]:
df = pd.read_csv('iwatebike.csv', encoding='cp932')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             291 non-null    object
 1   手口             291 non-null    object
 2   管轄警察署（発生地）     291 non-null    object
 3   管轄交番・駐在所（発生地）  291 non-null    object
 4   市区町村コード（発生地）   291 non-null    int64 
 5   都道府県（発生地）      291 non-null    object
 6   市区町村（発生地）      291 non-null    object
 7   町丁目（発生地）       288 non-null    object
 8   発生年月日（始期）      291 non-null    object
 9   発生時（始期）        291 non-null    object
 10  発生場所           291 non-null    object
 11  発生場所の詳細        291 non-null    object
 12  被害者の年齢         291 non-null    object
 13  被害者の職業         291 non-null    object
 14  施錠関係           291 non-null    object
dtypes: int64(1), object(14)
memory usage: 34.2+ KB


In [8]:
time = df['発生時（始期）'].value_counts().reset_index()
time

,発生時（始期）,count
0,18,34
1,7,33
2,16,25
3,8,23
4,19,20
5,12,17
6,20,16
7,9,14
8,15,12
9,17,12


In [9]:
time.columns = ['発生時（始期）','件数']
time

,発生時（始期）,件数
0,18,34
1,7,33
2,16,25
3,8,23
4,19,20
5,12,17
6,20,16
7,9,14
8,15,12
9,17,12


In [12]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='seaborn')
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [13]:
def geocode(address):

    # ジオコーティングURL
    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    # try/exceptでエラーをキャッチ
    # 成功の場合
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(url + s_quote)
        if len(response.json())>0:
            return response.json()[0]["geometry"]["coordinates"] 
        else:
            return False
        
    # 失敗の場合
    except:
        return False

In [15]:

df['住所'] = ''

df['lat'] = pd.Series(dtype=float)
df['lon'] = pd.Series(dtype=float)

In [16]:
df['住所'] = df['都道府県（発生地）']+df['市区町村（発生地）']+df['町丁目（発生地）']

In [17]:
df.sample(3)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係,住所,lat,lon
212,窃盗,自転車盗,一関,一関駅前,32093,岩手県,一関市,大手町,2021/3/7,11,その他,駐車（輪）場,10歳代,中学生,施錠せず,岩手県一関市大手町,NaN,NaN
60,窃盗,自転車盗,盛岡東,仙北町,32018,岩手県,盛岡市,西仙北１丁目,2021/7/6,8,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,10歳代,その他,施錠せず,岩手県盛岡市西仙北１丁目,NaN,NaN
241,窃盗,自転車盗,宮古,磯鶏,32026,岩手県,宮古市,磯鶏沖,2021/1/28,12,一戸建住宅,その他,10歳代,高校生,施錠せず,岩手県宮古市磯鶏沖,NaN,NaN


In [19]:
for i,row in df.iterrows():

    # ジオコーティング成功
    if geocode(row['住所']) != False:
        # 座標を変数に
        lon = geocode(row['住所'])[0]
        lat = geocode(row['住所'])[1]

        # データフレームに値をインプット        
        df.loc[i,'lon'] = lon
        df.loc[i,'lat'] = lat

        # 結果をprint
        print(row['住所'],lon,lat)
    
    # ジオコーティング失敗
    else:
        print(row['住所'],'ジオコーティング失敗')
        continue

岩手県盛岡市向中野２丁目 141.138977 39.682274
岩手県盛岡市向中野２丁目 141.138977 39.682274
岩手県盛岡市向中野４丁目 141.145569 39.678047
岩手県盛岡市本宮５丁目 141.122437 39.690464
岩手県盛岡市本宮７丁目 141.126511 39.68343
岩手県盛岡市本宮７丁目 141.126511 39.68343
岩手県盛岡市上田１丁目 141.144913 39.712486
岩手県盛岡市上田１丁目 141.144913 39.712486
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田３丁目 141.137741 39.71582
岩手県盛岡市上田４丁目 141.13501 39.72076
岩手県盛岡市上田４丁目 141.13501 39.72076
岩手県盛岡市上田４丁目 141.13501 39.72076
岩手県盛岡市上田４丁目 141.13501 39.72076
岩手県盛岡市上田４丁目 141.13501 39.72076
岩手県盛岡市高松１丁目 141.142212 39.722427
岩手県盛岡市高松１丁目 141.142212 39.722427
岩手

In [26]:
df=df.dropna()

In [20]:
center_lat = df.lat.mean()
center_lon = df.lon.mean()

In [27]:
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)
 
# kashiwaのデータフレームをループしてマーカーを作る
for index, row in df.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m